# Workshop Tutorial: General Concepts & Bundle Basics

In this first tutorial, we'll learn about the basics of the Bundle - which is the container of all Parameters within PHOEBE.  We'll see how to access individual Parameter and view and set their values.


This interactive workshop tutorial covers many of the same topics as the corresponding online tutorials:

* [The PHOEBE Bundle](http://phoebe-project.org/docs/2.3/tutorials/general_concepts.ipynb)
* [Advanced: Parameter Types](http://phoebe-project.org/docs/2.3/tutorials/parameters.ipynb)
* [Advanced: Parameter Units](http://phoebe-project.org/docs/2.3/tutorials/units.ipynb)
* [Advanced: Building a System](http://phoebe-project.org/docs/2.3/tutorials/contact_binary_hierarchy.ipynb)
* [Advanced: Contact Binary Hierarchy](http://phoebe-project.org/docs/2.3/tutorials/contact_binary_hierarchy.ipynb)
* [Advanced: Semi-Detached Systems](http://phoebe-project.org/docs/2.3/tutorials/requiv_crit_semidetached.ipynb)
* [Advanced: Saving, Loading, and Exporting](http://phoebe-project.org/docs/2.3/tutorials/saving_and_loading.ipynb)

# Setup

In [1]:
import phoebe
from phoebe import u,c

In [2]:
logger = phoebe.logger(clevel='WARNING')

# Default Binary Bundle

Everything for our system will be stored in a single Python object that we call the "Bundle".  Let's start just by creating the default binary system and store it in a Bundle-object which we'll call "b" (short for bundle).

In [3]:
b = phoebe.default_binary()

The Bundle is just a collection of Parameter objects along with some callable methods.  Here we can see that the default binary Bundle consists of over 100 individual parameters.

In [4]:
b

<PHOEBE Bundle: 135 parameters | contexts: setting, system, constraint, component, compute, figure>

If we want to view or edit a Parameter in the Bundle, we first need to know how to access it.  Each Parameter object has a number of tags which can be used to filter (similar to a database query).  When filtering the Bundle, a ParameterSet is returned - this is essentially just a subset of the Parameters in the Bundle and can be further filtered until eventually accessing a single Parameter.

In [5]:
b.filter(context='compute')

<ParameterSet: 17 parameters | components: secondary, primary>

Here we filtered on the context tag for all Parameters with context='compute' (i.e. the options for computing a model).  If we want to see all the available options for this tag in the Bundle, we can use the plural form of the tag.

In [6]:
b.contexts

['system', 'component', 'constraint', 'figure', 'setting', 'compute']

Although there is no strict hierarchy to the tags, it can be helpful to think of the context tag as the top-level tag and is often very helpful to filter by the appropriate context first.

Other tags currently include:
* kind
* component
* dataset
* model
* time
* qualifier

Accessing the plural form of the tag as an attribute also works on a filtered ParameterSet

In [7]:
b.filter(context='compute').components

['primary', 'secondary']

This then tells us what can be used to filter further.

In [8]:
b.filter(context='compute').filter(component='primary')

<ParameterSet: 4 parameters | qualifiers: ntriangles, mesh_method, distortion_method, atm>

The qualifier tag is the shorthand name of the Parameter itself.  If you don't know what you're looking for, it is often useful to list all the qualifiers of the Bundle or a given ParameterSet.

In [9]:
b.filter(context='compute', component='primary').qualifiers

['mesh_method', 'ntriangles', 'distortion_method', 'atm']

Now that we know the options for qualifier within this filter, we can choose to filter on one of those.  Let's filter by the `ntriangles` qualifier.

In [10]:
b.filter(context='compute', component='primary', qualifier='ntriangles')

<ParameterSet: 1 parameters>

Once we filter far enough to get to a single Parameter, we can use get_parameter to return the Parameter object itself (instead of a ParameterSet).

In [11]:
b.filter(context='compute', component='primary', qualifier='ntriangles').get_parameter()

<Parameter: ntriangles=1500 | keys: description, value, limits, visible_if, copy_for, readonly, advanced, latexfmt>

As a shortcut, get_parameter also takes filtering keywords.  So the above line is also equivalent to the following:

In [12]:
b.get_parameter(context='compute', component='primary', qualifier='ntriangles')

<Parameter: ntriangles=1500 | keys: description, value, limits, visible_if, copy_for, readonly, advanced, latexfmt>

Each Parameter object contains several keys that provide information about that Parameter.  The keys "description" and "value" are always included, with additional keys available depending on the type of Parameter.

In [13]:
b.get_parameter(context='compute', component='primary', qualifier='ntriangles').get_value()

1500

In [14]:
b.get_parameter(context='compute', component='primary', qualifier='ntriangles').get_description()

"Requested number of triangles (won't be exact)."

Since the Parameter for `ntriangles` is an IntegerParameter, it also includes a key for the allowable limits.

In [15]:
b.get_parameter(context='compute', component='primary', qualifier='ntriangles').get_limits()

[100, None]

In this case, we're looking at the Parameter called `ntriangles` with the component tag set to 'primary'.  This Parameter therefore defines how many triangles should be created when creating the mesh for the star named 'primary'.  By default, this is set to 1500 triangles, with allowable values above 100.

If we wanted a finer mesh, we could change the value.

In [16]:
b.get_parameter(context='compute', component='primary', qualifier='ntriangles').set_value(2000)

In [17]:
b.get_parameter(context='compute', component='primary', qualifier='ntriangles')

<Parameter: ntriangles=2000 | keys: description, value, limits, visible_if, copy_for, readonly, advanced, latexfmt>

If we choose the `distortion_method` qualifier from that same ParameterSet, we'll see that it has a few different keys in addition to description and value.

In [18]:
b.get_parameter(context='compute', component='primary', qualifier='distortion_method')

<Parameter: distortion_method=roche | keys: description, choices, value, visible_if, copy_for, readonly, advanced, latexfmt>

In [19]:
b.get_parameter(context='compute', component='primary', qualifier='distortion_method').get_value()

'roche'

In [20]:
b.get_parameter(context='compute', component='primary', qualifier='distortion_method').get_description()

'Method to use for distorting stars'

Since the `distortion_method` Parameter is a ChoiceParameter, it contains a key for the allowable choices.

In [21]:
b.get_parameter(context='compute', component='primary', qualifier='distortion_method').get_choices()

['roche', 'rotstar', 'sphere', 'none']

We can only set a value if it is contained within this list - if you attempt to set a non-valid value, an error will be raised.

In [22]:
b.get_parameter(context='compute', component='primary', qualifier='distortion_method').set_value('rotstar')

In [23]:
b.get_parameter(context='compute', component='primary', qualifier='distortion_method').get_value()

'rotstar'

Parameter types include:
* String Parameter
* Choice Parameter
* Select Parameter
* Float Parameter
* Integer Parameter
* Boolean Parameter
* FloatArray Parameter

these Parameter types and their available options are all described in great detail in [Advanced: Parameter Types](http://phoebe-project.org/docs/2.3/tutorials/parameters.ipynb)

## Twigs

As a shortcut to needing to filter by all these tags, the Bundle and ParameterSets can be filtered through what we call "twigs" (as in a Bundle of twigs).  These are essentially a single string-representation of the tags, separated by @ symbols.

This is very useful as a shorthand when working in an interactive Python console, but somewhat obfuscates the names of the tags and can make it difficult if you use them in a script and make changes earlier in the script.

For example, the following lines give identical results:

In [24]:
b.filter(context='compute', component='primary')

<ParameterSet: 4 parameters | qualifiers: ntriangles, mesh_method, distortion_method, atm>

In [25]:
b['primary@compute']

<ParameterSet: 4 parameters | qualifiers: ntriangles, mesh_method, distortion_method, atm>

In [26]:
b['compute@primary']

<ParameterSet: 4 parameters | qualifiers: ntriangles, mesh_method, distortion_method, atm>

However, this dictionary-style twig access will never return a ParameterSet with a single Parameter, instead it will return the Parameter itself.  This can be seen in the different output between the following two lines:

In [27]:
b.filter(context='compute', component='primary', qualifier='distortion_method')

<ParameterSet: 1 parameters>

In [28]:
b['distortion_method@primary@compute']

<Parameter: distortion_method=rotstar | keys: description, choices, value, visible_if, copy_for, readonly, advanced, latexfmt>

This dictionary-style access can also set the value directly:

In [29]:
b['distortion_method@primary@compute'] = 'roche'

In [30]:
print(b['distortion_method@primary@compute'])

Parameter: distortion_method@primary@phoebe01@compute
                       Qualifier: distortion_method
                     Description: Method to use for distorting stars
                           Value: roche
                         Choices: roche, rotstar, sphere, none
                  Constrained by: 
                      Constrains: None
                      Related to: None
                 Only visible if: mesh_method:marching,hierarchy.is_meshable:true



And can even provide direct access to the keys/attributes of the Parameter (value, description, limits, etc)

In [31]:
print(b['value@distortion_method@primary@compute'])

roche


In [32]:
print(b['description@distortion_method@primary@compute'])

Method to use for distorting stars


As with the tags, you can call .twigs on any ParameterSet to see the "smallest unique twigs" of the contained Parameters

In [33]:
b['compute'].twigs

['sample_from@phoebe01@phoebe@compute',
 'comments@phoebe01@phoebe@compute',
 'use_server@phoebe01@phoebe@compute',
 'dynamics_method@phoebe01@phoebe@compute',
 'ltte@phoebe01@phoebe@compute',
 'irrad_method@phoebe01@phoebe@compute',
 'boosting_method@phoebe01@phoebe@compute',
 'eclipse_method@phoebe01@phoebe@compute',
 'horizon_method@phoebe01@phoebe@compute',
 'mesh_method@primary@phoebe01@phoebe@compute',
 'mesh_method@secondary@phoebe01@phoebe@compute',
 'ntriangles@primary@phoebe01@phoebe@compute',
 'ntriangles@secondary@phoebe01@phoebe@compute',
 'distortion_method@primary@phoebe01@phoebe@compute',
 'distortion_method@secondary@phoebe01@phoebe@compute',
 'atm@primary@phoebe01@phoebe@compute',
 'atm@secondary@phoebe01@phoebe@compute']

Since the more verbose method without twigs is a bit clearer to read, most of the tutorials will show that syntax, but feel free to use twigs if they make more sense to you

To see a nice representation of the names (twigs or qualifier) of all the parameters and their descriptions, we can call `info` on any bundle or ParameterSet.

In [34]:
print(b.info)

ParameterSetInfo: (qualfier/twig: description)
                              Av: Extinction Av
                              Rv: Extinction law parameter
                            abun: Abundance/Metallicity
    asini@binary@orbit@component: Projected semi major axis
                asini@constraint: expression that determines the constraint
            asini@star@component: Projected semi major axis of the component in the orbit
                             atm: Atmosphere table
                 auto_add_figure: Whether to automatically add figure parameters when a dataset is added with a new dataset type, or a solution is added.
              auto_remove_figure: Whether to automatically remove figure parameters when the referenced dataset/solution are removed.
                 boosting_method: Type of boosting method
                           color: Color to use for figures in which color_source is set to component
                        comments: User-provided comments for these

In [35]:
print(b.filter(context='component').info)

ParameterSetInfo: (qualfier/twig: description)
                            abun: Abundance/Metallicity
    asini@binary@orbit@component: Projected semi major axis
            asini@star@component: Projected semi major axis of the component in the orbit
                            dpdt: Time derivative of orbital period (anomalistic), where period is defined at t0@system
                          dperdt: Time derivative of argument of periastron, where per0 is defined at t0@system
                             ecc: Eccentricity
                           ecosw: Eccentricity times cos of argument of periastron
                           esinw: Eccentricity times sin of argument of periastron
     freq@binary@orbit@component: Orbital frequency (sidereal)
             freq@star@component: Rotation frequency (wrt the sky)
                       gravb_bol: Bolometric gravity brightening
     incl@binary@orbit@component: Orbital inclination angle
             incl@star@component: Inclination o

## Units

Each FloatParameter has an associated unit. Let's look at the 'sma' Parameter for the binary orbit.

In [36]:
b.get_parameter(qualifier='sma', component='binary', context='component')

<Parameter: sma=5.3 solRad | keys: description, value, quantity, default_unit, limits, visible_if, copy_for, readonly, advanced, latexfmt>

From the representation above, we can already see that the units are in solar radii. We can access the units directly via get_default_unit.

In [37]:
b.get_parameter(qualifier='sma', component='binary', context='component').get_default_unit()

Unit("solRad")

Calling get_value returns only the float of the value in these units.

In [38]:
b.get_parameter(qualifier='sma', component='binary', context='component').get_value()

5.3

Alternatively, you can access an astropy quantity object that contains the value and unit by calling get_quantity.

In [39]:
b.get_parameter(qualifier='sma', component='binary', context='component').get_quantity()

<Quantity 5.3 solRad>

Both get_value and get_quantity also accept a unit argument which will return the value or quantity in the requested units (if able to convert). This unit argument takes either a unit object (we imported a forked version of astropy units from within PHOEBE) or a string representation that can be parsed.

In [40]:
b.get_parameter(qualifier='sma', component='binary', context='component').get_value(unit=u.km)

3687210.0

If for some reason you want to change the default units, you can, but just be careful that this could cause some float-point precision issue

In [41]:
b.get_parameter(qualifier='sma', component='binary', context='component').set_default_unit('mm')

In [42]:
b.get_parameter(qualifier='sma', component='binary', context='component').get_quantity()

<Quantity 3.68721e+12 mm>

In [43]:
b.get_parameter(qualifier='sma', component='binary', context='component').get_quantity(unit='solRad')

<Quantity 5.3 solRad>

## Saving & Loading

The entire bundle object can be saved to (and reloaded from) and ASCII file.

In [44]:
b.save('test.phoebe')

'test.phoebe'

In [45]:
b = phoebe.load('test.phoebe')

# Exercise

Find and access the value of the effective temperature of the primary star via filtering and twig access.

Find the choices for the `atm` Parameter

Find what the `ltte` Parameter stands for.  Does it have choices?

Change the default unit of the argument of periastron from degrees to radians and print the result with the new units.

Set the default_unit for all radii and smas of all components to AU and then show their values. (Hint: you can also call set_default_unit_all to act on a ParameterSet instead of a Parameter)

Find and set the following Parameters:
* effective temperature of the secondary star to 5500 K
* inclination of the binary to 86 degrees

You likely noticed that there are several (5!) Parameters in the Bundle for inclination.  This is because there is an inclination for the orbit as well as for each of the two stars in the binary system.  The other 2 are called Constraints which relate these Parameters to each other... which will be the topic of the next tutorial.